In [1]:
import numpy as np
from math import exp, log, sqrt
from datetime import datetime

In [3]:
print("Introduce el valor del subyacente: ", end="")
suby = float(input())

print("Introduce el plazo en días: ", end="")
plazo = int(input())

print("Introduce los puntos forward: ", end="")
ptfw = float(input())

print("Introduce el nocional del forward: ", end="")
nocf = float(input())

print("Introduce la tasa USD (simple) (%): ", end="")
UsdSimple = float(input()) / 100

print("Introduce la volatilidad (%): ", end="")
vol = float(input()) / 100

print("Introduce la TIIE a 3 meses (para factor de descuento): ", end="")
tiie = float(input()) / 100

print("Introduce la fecha de valuación del bono (dd/mm/aaaa): ", end="")
fval = input()

print("Introduce el nocional del bono: ", end="")
nocb = float(input())

print("Introduce la fecha de vencimiento del bono (dd/mm/aaaa): ", end="")
fven = input()

print("Introduce la tasa cupón (%): ", end="")
tcup = float(input()) / 100

print("Introduce la YTM (%): ", end="")
ytm = float(input()) / 100

print("Introduce la tasa interbancaria (%): ", end="")
tibanc = float(input()) / 100

print("Introduce la tasa de recuperación en incumplimiento (%): ", end="")
recup = float(input()) / 100

print("Introduce los días para cálculo de probabilidad de incumplimiento: ", end="")
dinc = int(input())

fval_d = int(fval[:2])
fval_m = int(fval[3:5])
fval_a = int(fval[6:])

fven_d = int(fven[:2])
fven_m = int(fven[3:5])
fven_a = int(fven[6:])

Introduce el valor del subyacente: 21.77
Introduce el plazo en días: 90
Introduce los puntos forward: 4118
Introduce el nocional del forward: 10000000
Introduce la tasa USD (simple) (%): 0.78
Introduce la volatilidad (%): 20
Introduce la TIIE a 3 meses (para factor de descuento): 7.13
Introduce la fecha de valuación del bono (dd/mm/aaaa): 11/04/2020
Introduce el nocional del bono: 100
Introduce la fecha de vencimiento del bono (dd/mm/aaaa): 15/03/2028
Introduce la tasa cupón (%): 5
Introduce la YTM (%): 7
Introduce la tasa interbancaria (%): 0.67
Introduce la tasa de recuperación en incumplimiento (%): 20
Introduce los días para cálculo de probabilidad de incumplimiento: 90


In [4]:
pfwd = suby + ptfw / 10000
UsdCont = log(1 + UsdSimple * plazo / 360) * 360 / plazo
MxnSimple = (pfwd / suby * (1 + UsdSimple * plazo / 360) - 1) * 360 / plazo
MxnCont = log(1 + MxnSimple * plazo / 360) * 360 / plazo

In [5]:
n = 1000
np.random.seed(12345)
norm = np.random.normal(0, 1, (n, plazo))
norm = np.exp(norm * sqrt(1/360) * vol + (MxnCont - UsdCont - 0.5 * vol ** 2) / 360)
norm[:,0] = norm[:,0] * suby
for i in range(plazo - 1):
    norm[:,i + 1] = norm[:,i + 1] * norm[:,i]

In [6]:
mtm = norm
for i in range(plazo):
    mtm[:,i] = nocf * (norm[:,i] * exp((MxnCont - UsdCont) * (plazo - i - 1) / 360) - pfwd) * exp(-MxnCont * (plazo - i - 1) / 360)
epe = np.nanmean(np.where(mtm>0, mtm, np.nan), axis=0)
pfe = []
for i in range(plazo):   
    pfe.append(np.percentile(np.array([x for x in mtm[:,i] if x > 0]), 95))
pfe = np.array(pfe)

In [7]:
fval = datetime(fval_a, fval_m, fval_d)
fven = datetime(fven_a, fven_m, fven_d)

#Aproximación de diferencia con años de 360 días
dif_dias = (fven - fval).days - 5 * ((fven - fval).days // 365)
per = dif_dias // 180

pinc = 0.15

dacup = []
factd = []
factd_int = []
for i in range(per + 1):
    if dif_dias - 180 * per + 180 * i > 0:
        dacup.append(dif_dias - 180 * per + 180 * i)
    factd.append(1 / ((1 + ytm * 0.5) ** (dacup[i] / 180)))
    factd_int.append(1 / ((1 + tibanc * 0.5) ** (dacup[i] / 180)))
    
flu = [tcup * 0.5 * nocb] * per
flu.append(nocb + tcup * 0.5 * nocb)

In [8]:
def GoalSeek(fun,goal,x0,fTol=0.0001,MaxIter=1000):
    # Goal Seek function of Excel
    #   via use of Line Search and Bisection Methods

    # Inputs
    #   fun     : Function to be evaluated
    #   goal    : Expected result/output
    #   x0      : Initial estimate/Starting point

    # Initial check
    if fun(x0)==goal:
        print('Exact solution found')
        return x0

    # Line Search Method
    step_sizes=np.logspace(-1,4,6)
    scopes=np.logspace(1,5,5)

    vFun=np.vectorize(fun)

    for scope in scopes:
        break_nested=False
        for step_size in step_sizes:

            cApos=np.linspace(x0,x0+step_size*scope,int(scope))
            cAneg=np.linspace(x0,x0-step_size*scope,int(scope))

            cA=np.concatenate((cAneg[::-1],cApos[1:]),axis=0)

            fA=vFun(cA)-goal

            if np.any(np.diff(np.sign(fA))):

                index_lb=np.nonzero(np.diff(np.sign(fA)))

                if len(index_lb[0])==1:

                    index_ub=index_lb+np.array([1])

                    x_lb=np.asscalar(np.array(cA)[index_lb][0])
                    x_ub=np.asscalar(np.array(cA)[index_ub][0])
                    break_nested=True
                    break
                else: # Two or more roots possible

                    index_ub=index_lb+np.array([1])

                    print('Other solution possible at around, x0 = ', np.array(cA)[index_lb[0][1]])

                    x_lb=np.asscalar(np.array(cA)[index_lb[0][0]])
                    x_ub=np.asscalar(np.array(cA)[index_ub[0][0]])
                    break_nested=True
                    break

        if break_nested:
            break
    if not x_lb or not x_ub:
        print('No Solution Found')
        return

    # Bisection Method
    iter_num=0
    error=10

    while iter_num<MaxIter and fTol<error:
        
        x_m=(x_lb+x_ub)/2
        f_m=fun(x_m)-goal

        error=abs(f_m)

        if (fun(x_lb)-goal)*(f_m)<0:
            x_ub=x_m
        elif (fun(x_ub)-goal)*(f_m)<0:
            x_lb=x_m
        elif f_m==0:
            print('Exact spolution found')
            return x_m
        else:
            print('Failure in Bisection Method')
        
        iter_num+=1

    return x_m

def risk_p(pinc):
    prob_cump = [1]
    for i in range(per + 1):
        prob_cump.append((1 - pinc) ** (i + 1))
    return (np.array(flu) * np.array(factd_int) * np.array(prob_cump[1:])).sum() + (np.array(factd_int) * np.array(prob_cump[:-1])).sum() * nocb * recup * pinc

In [9]:
dir_p = (np.array(flu) * np.array(factd)).sum()
pinc_opt = GoalSeek(risk_p,dir_p,pinc)
pinc_acum = 1 - ((1 - pinc_opt) ** (dinc / 180))

prob_cond = []
factd_tiie = []
for i in range(plazo):
    prob_cond.append((1 - pinc_opt) ** ((i) / 180) - (1 - pinc_opt) ** ((i + 1) / 180))
    factd_tiie.append(1 / ((1 + tiie * 91 / 360) ** ((i + 1) / 91)))

cva_epe = (epe * np.array(factd_tiie) * np.array(prob_cond)).sum()
cva_pfe = (pfe * np.array(factd_tiie) * np.array(prob_cond)).sum()

In [10]:
print(f"La probabilidad de incumplimiento acumulada para {dinc} días es de: {round(pinc_acum * 100,2)}%")
print(f"El CVA EPE es de: {round(cva_epe,0)}")
print(f"El CVA PFE es de: {round(cva_pfe,0)}")

La probabilidad de incumplimiento acumulada para 90 días es de: 1.94%
El CVA EPE es de: 225419.0
El CVA PFE es de: 565263.0
